# Channel 실습

In [7]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [2]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    val controlChannel = Channel<Int>()
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(400); channel.send(2 * x); }
        controlChannel.send(0) // 감독관한테 나 일 다했어
        println("sending even number done")
    }
    launch { // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..5) { delay(800); channel.send(2 * x - 1) }
        channel.close()
        controlChannel.send(1) // 감독관한테 나 일 다했어
        println("sending odd number done")
    }
    launch { // 감독관 작업?
        controlChannel.receive()
        controlChannel.receive()
        channel.close()
        controlChannel.close()
    }
    // here we print five received integers:
    for(x in 1..10) { println(channel.receive()) }
    println("Done!")
}

2
1
4
6
3
8
sending even number done
10
5
7
sending odd number done
9
Done!


In [3]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

runBlocking {
    val channel = Channel<Int>()
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) { delay(400); channel.send(x) }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) { println(x) }
    // for(x in 1..5) { println(channel.receive()) }
    println("Done!")
}

1
2
3
4
5
6
7
8
9
10
Done!


In [4]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

val channel = Channel<Int>()

runBlocking {
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) { channel.send(x) }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) { delay(400); println(x) }
    println("Done!")
}

1
2
3
4
5
6
7
8
9
10
Done!


In [5]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlinx.coroutines.flow.*

// 채널 안에 메모리가 허락하는 한 무한정의 버퍼를 두어 데이터를 많이 쌓아둘 수 있음 
val channel = Channel<Int>( Channel.UNLIMITED )
val count = MutableStateFlow<Int>(0) // for MT-safe(multi-thread safe) counter

runBlocking {
    
    launch {
        // this might be heavy CPU-consuming computation or async logic, we'll just send five squares
        for (x in 1..10) {
            // delay(100); // 주석을 처리했다 풀었다, 그리고 50 으로도 해보라
            channel.send(x)
            count.update { it -> it+1 }
            println("count ${count.value}")
        }
        channel.close();
    }
    
    // here we print five received integers:
    for (x in channel) {
        /// delay(100); // 주석을 처리했다 풀었다, 그리고 50 으로도 해보라
        count.update { it -> it-1 }
        println("received ${x}, channel has ${count.value} items")
    }
    // for(x in 1..5) { println(channel.receive()) }
    println("Done!")
}

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
received 1, channel has 9 items
received 2, channel has 8 items
received 3, channel has 7 items
received 4, channel has 6 items
received 5, channel has 5 items
received 6, channel has 4 items
received 7, channel has 3 items
received 8, channel has 2 items
received 9, channel has 1 items
received 10, channel has 0 items
Done!


In [6]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*
import kotlinx.coroutines.flow.*

val chan0 = Channel<Int>( Channel.UNLIMITED ) // 카드게임에서 deck의 역할에 해당
val cnt0 = MutableStateFlow<Int>(0)
val hand0 = ArrayDeque( listOf(10,11,12,13) )

val chan1 = Channel<Int>( Channel.UNLIMITED ) // 카드게임에서 deck의 역할에 해당
val cnt1 = MutableStateFlow<Int>(0)
val hand1 = ArrayDeque( listOf(20,21,22,23) )

runBlocking {
    for (i in 0..9) chan0.send(i) // 10개의 정수를 chan0에 채워놓고 시작
    cnt0.update { it -> 9 }
    
    launch { // player 0
        for (i in 0..9) {
            hand0.addFirst( chan0.receive() )
            cnt0.update { it -> it-1 }.also { println("cnt0--") }
            delay(150) // 다음에는 이걸 200으로, 그리고 
            chan1.send( hand0.removeLast() )
            cnt1.update { it -> it+1 }.also { println("cnt1++") }
        }
    }
    
    launch { // player 1
        for (i in 0..9) {
            hand1.addFirst( chan1.receive() )
            cnt1.update { it -> it-1 }.also { println("cnt1--") }
            delay(200) // 반대로 이걸 150으로 바꿔서 실습해 보라
            chan0.send( hand1.removeLast() )
            cnt0.update { it -> it+1 }.also { println("cnt0++") }
            println("cnt0++")
        }
    }
    
    for (i in 0..25) { // 일정 시간 간격으로 각 채널에 들어있는 데이터 개수를 출력
        delay(100)
        println("cnt0: ${cnt0.value}, cnt1:${cnt1.value}")
    }

    println("Done!")
}

cnt0--
cnt0: 8, cnt1:0
cnt1++
cnt0--
cnt1--
cnt0: 7, cnt1:0
cnt1++
cnt0--
cnt0: 6, cnt1:1
cnt0++
cnt0++
cnt1--
cnt0: 7, cnt1:0
cnt1++
cnt0--
cnt0: 6, cnt1:1
cnt0++
cnt0++
cnt1--
cnt1++
cnt0--
cnt0: 6, cnt1:1
cnt0: 6, cnt1:1
cnt1++
cnt0--
cnt0++
cnt0++
cnt1--
cnt0: 6, cnt1:1
cnt1++
cnt0--
cnt0: 5, cnt1:2
cnt0++
cnt0++
cnt1--
cnt0: 6, cnt1:1
cnt1++
cnt0--
cnt0: 5, cnt1:2
cnt0++
cnt0++
cnt1--
cnt1++
cnt0--
cnt0: 5, cnt1:2
cnt0: 5, cnt1:2
cnt1++
cnt0--
cnt0++
cnt0++
cnt1--
cnt0: 5, cnt1:2
cnt1++
cnt0: 5, cnt1:3
cnt0++
cnt0++
cnt1--
cnt0: 6, cnt1:2
cnt0: 6, cnt1:2
cnt0++
cnt0++
cnt1--
cnt0: 7, cnt1:1
cnt0: 7, cnt1:1
cnt0++
cnt0++
cnt1--
cnt0: 8, cnt1:0
cnt0: 8, cnt1:0
cnt0++
cnt0++
cnt0: 9, cnt1:0
cnt0: 9, cnt1:0
cnt0: 9, cnt1:0
cnt0: 9, cnt1:0
cnt0: 9, cnt1:0
Done!


In [8]:
DISPLAY( HTML("hello0").withId("player0") )
DISPLAY( HTML("hello1").withId("player1") )

z

Z

In [9]:
import kotlinx.coroutines.*
/*
val d0 = CGameDeck(..)
val d1 = CGameDeck(..)
val p0 = CCPlayer(..)
val p1 = CCPlayer(..)
*/
runBlocking {
    launch {
        for (x in 'a'..'z') {
            // p0.draw(d0)
            // delay(200)
            // UPDATE_DISPLAY( HTML(x.toString()), "player0")
            // p0.put(d1)
            delay(200)
            UPDATE_DISPLAY( HTML(x.toString()), "player0")
        }
    }

    launch {
        for (x in 'A'..'Z') {
            // p1.draw(d1)
            // delay(200)
            // UPDATE_DISPLAY( HTML(x.toString()), "player1")
            // p1.put(d0)
            delay(150)
            UPDATE_DISPLAY( HTML(x.toString()), "player1")
        }
    }
    
    // 메인 작업이 바로 게임 마스터
    delay(5000)
    println("Done!")
}

Done!
